In [1]:
import os.path
from subprocess import check_call
import importlib
import os
import sys

modules = ["findspark","pyspark"]

def check_modules(modules, upgrade=False):
    print("Checking required modules")
    for m in modules:
        torch_loader = importlib.util.find_spec(m)
        if torch_loader is not None and not upgrade:
            print(m," found")
        else:
            if upgrade:
                print("upgrading ",m)
            else:
                print(m," not found, installing")
            if 'google.colab' in sys.modules:
                if upgrade:
                    check_call(["pip", "install", "--upgrade", m])
                else:
                    check_call(["pip", "install", "-q", m])
            else:
                if upgrade:
                    check_call([sys.executable, "-m", "pip", "install", "--user", "--upgrade", m])
                else:
                    check_call([sys.executable, "-m", "pip", "install", "--user", m])


if 'google.colab' in sys.modules:
    !apt-get install openjdk-8-jdk-headless -qq > /dev/null
    spark = "spark-3.2.0-bin-hadoop3.2.tgz"
    if not os.path.isfile(spark):
        !wget -q https://downloads.apache.org/spark/spark-3.2.0/{spark}
        !tar xf {spark}
        os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
        os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"
        
check_modules(modules)
print("Done!")






Checking required modules
findspark  found
pyspark  found
Done!


In [2]:
%matplotlib inline
import os
# cambiamos las variables del sistema
spark = 'C:/ucm-master-iot/TDM/spark/spark-3.1.2-bin-hadoop3.2'
#spark = 'C:/hlocal/tdm/spark/spark-3.2.0-bin-hadoop3.2'
if not(os.path.isdir(spark+"/bin")) or not(os.path.isdir(spark+"/jars")) :
        print("Error, la carpeta en 'spark' debe contener los directorios bin y jars ")
else:    
        # en el path se añade
        #path = os.environ.get('PATH') 
        #path = path+ ';'+spark+'\\bin;'
        #os.environ['PATH'] = path
        os.environ['SPARK_HOME']= spark 
        os.environ['HADOOP_HOME']= spark 
        os.environ['PYSPARK_DRIVER_PYTHON']= 'jupyter'
        os.environ['PYSPARK_DRIVER_PYTHON_OPTS']='notebook'

        # si da problema con collect quizás haya que poner java_home a la localización de java 8
        #os.environ['JAVA_HOME']= 'C:\\Program Files\\Java\\jdk1.8.0_151'
        #os.environ['PATH'] = os.environ.get('JAVA_HOME')+'\\bin;'+spark
        print("Hecho")
print("Preparado!!")

Hecho
Preparado!!


In [3]:
import findspark
findspark.init()

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() # SparkSession.builder.getOrCreate()
sc = spark.sparkContext


df = spark.sql('''select 'spark' as hi ''')
df.show()

+-----+
|   hi|
+-----+
|spark|
+-----+



In [4]:
from pyspark.ml.fpm import FPGrowth

df = spark.createDataFrame([
    (0, ['A','B','C','D']),
    (1, ['A','B','D']),
    (2, ['A','B']),
    (3, ['B','C','D']),
    (4, ['B','C']),
    (5, ['C','D']),
    (6, ['B','D'])
], ["id", "items"])


fpGrowth = FPGrowth(itemsCol="items", minSupport=0.4, minConfidence=0.7)
model = fpGrowth.fit(df)

# Display frequent itemsets.
model.freqItemsets.show()

# Display generated association rules.
model.associationRules.show()

# transform examines the input items against all the association rules
# and summarize the consequents as prediction
model.transform(df).show()

+------+----+
| items|freq|
+------+----+
|   [B]|   6|
|   [D]|   5|
|[D, B]|   4|
|   [C]|   4|
|[C, D]|   3|
|[C, B]|   3|
|   [A]|   3|
|[A, B]|   3|
+------+----+

+----------+----------+----------+------------------+-------------------+
|antecedent|consequent|confidence|              lift|            support|
+----------+----------+----------+------------------+-------------------+
|       [C]|       [D]|      0.75|              1.05|0.42857142857142855|
|       [C]|       [B]|      0.75|             0.875|0.42857142857142855|
|       [A]|       [B]|       1.0|1.1666666666666667|0.42857142857142855|
|       [D]|       [B]|       0.8|0.9333333333333335| 0.5714285714285714|
+----------+----------+----------+------------------+-------------------+

+---+------------+----------+
| id|       items|prediction|
+---+------------+----------+
|  0|[A, B, C, D]|        []|
|  1|   [A, B, D]|        []|
|  2|      [A, B]|        []|
|  3|   [B, C, D]|        []|
|  4|      [B, C]|       [D]